In [ ]:
from transformers import BertForMaskedLM, BertTokenizer
from data_dir import pretrained_3kmer_dir
from data_preparation import str_kmer, kmer
from torch import tensor

tokenizer = BertTokenizer.from_pretrained(pretrained_3kmer_dir)
sequences = ["ATGC" * 128, "GACT" * 130]
sequences_arr = [str_kmer(s, 3) for s in sequences]
# print(sequences_arr[1])

long_sequence = sequences[1]
long_sequences = kmer(long_sequence, 512)
long_sequences_kmer = [str_kmer(s, 3) for s in long_sequences]

tokenized = tokenizer.encode(long_sequences_kmer[1], add_special_tokens=True)
#print(len(long_sequences_kmer[1].split(' ')))
#print(tokenized)


def preprocess_sequence(seq, tokenizer, max_length=512, kmer_size=3):
    sequences = kmer(seq, max_length)
    tokenized_s_array = []
    segment_embeddings_array = []
    for s in sequences:
        # print(f"Processing {s}, length {len(s)}")
        s = kmer(s, 3)
        #tokenized_s = tokenizer.encode(s, add_special_tokens=True, return_tensors='pt')
        tokenized_s = tokenizer.encode_plus(s, add_special_tokens=True, return_tensors='pt')
        segment_embeddings = tensor([1 for token in s])
        tokenized_s_array.append(tokenized_s)
        segment_embeddings_array.append(segment_embeddings)

    return tokenized_s_array, segment_embeddings_array

tokenized_s_array, segment_embeddings_array = preprocess_sequence(sequences[1], tokenizer)
print(tokenized_s_array)
print(segment_embeddings_array)

In [64]:
len(long_sequences_kmer[1].split(' '))
result = tokenizer.encode_plus(long_sequences_kmer[0], add_special_tokens=True, return_tensors='pt')
result

{'input_ids': tensor([[ 2, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55,
         14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44, 33, 55, 14, 44,
         33, 5